# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기
# Face Detection vs Face Recognition

In [6]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): # 대상 이미지 (3채널), x, y, width, height, 덮어씌울 이미지 (4채널)
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 -> 255 로 나누면 0 ~ 1 사이의 값 (1 : 불투명, 0: 완전 투명)

    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))


# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
video_file = 'man.mp4'
cap = cv2.VideoCapture(video_file)

img_right = cv2.imread('./right_horn.png', cv2.IMREAD_UNCHANGED)
img_left = cv2.imread('./left_horn.png', cv2.IMREAD_UNCHANGED)
img_nose = cv2.imread('./nose.png', cv2.IMREAD_UNCHANGED)

# model_selection 촬영자가 5m 이내이면 0, 그 이상이면 1 (0으로 했는데 얼굴이 2개가 인식을 해서 1로 함)
# min_detection_confidence 얼굴의 확신을 가지는 확률 : 높을 수록 사람 얼굴을 잘 인식함
with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.9) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.detections:
            for detection in results.detections:
                # 얼굴에 있는 요소를 빨간점으로 표시
                # mp_drawing.draw_detection(image, detection)

                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝부분

                h, w, _ = image.shape

                # 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 40, int(right_eye.y * h) - 120)
                left_eye = (int(left_eye.x * w) + 40, int(left_eye.y * h) - 120)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (0, 0, 255), 10, cv2.LINE_AA)
                # cv2.circle(image, left_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
                # cv2.circle(image, nose_tip, 60, (0, 255, 255), 10, cv2.LINE_AA)

                # 각 특징에다가 이미지 그리기
                # image[right_eye[1] - 50 : right_eye[1]+50, right_eye[0]-50 : right_eye[0]+50] = img_right
                # image[left_eye[1] - 50 : left_eye[1]+50, left_eye[0]-50 : left_eye[0]+50] = img_left
                # image[nose_tip[1] - 50 : nose_tip[1]+50, nose_tip[0]-150 : nose_tip[0]+150] = img_nose

                overlay(image, *right_eye, 50, 50, img_right)
                overlay(image, *left_eye, 50, 50, img_left)
                overlay(image, *nose_tip, 150, 50, img_nose)

        # cv2.imshow('Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        cv2.imshow('Face Detection', image)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()